# Risk Data Generator _datav3

The purpose of this script is to convert simulated data (raw output) into risk scores and store these for each class of simulations each with their own parameter regimes. "Run all cells" will create a set of risk matrices stored in npz form in the './data/' folder. These matrices are then access for performance reporting. 

In [2]:
import numpy as np
import sys
import matplotlib
sys.path.append('../qslam')

from visualiserisk import *
from qslamdesignparams import GRIDDICT

#### Parameter Regimes

The set of parameters are given in the dictionary format, LOOPS_DICT, and used for analysis via ParamUpdater. 

In [3]:
# Choose defaults to match floor case (heights didn't work)
TRUTHKWARGS = {}

BARRIER_FLOOR = 0.25*np.pi
BARRIER_HEIGHT = 0.75*np.pi
FLOOR_RATIO = 0.4 # matches floor case

TRUTHKWARGS["OneStepdheight"] = {"low": BARRIER_FLOOR, 
                                 "high": BARRIER_HEIGHT} 
TRUTHKWARGS["OneStepdfloorarea"] = FLOOR_RATIO 

prefix_list = ['2019_Jun_1D', '2019_Jun_2D', '2019_Jun_2D_Gssn']


In [5]:
# ------------------------------------------------------------------------------
# TURN OFF PARAMETER SCANS
# ------------------------------------------------------------------------------

idx_prevar = 0 
# Fix truth configurations
meta_truth_floor_scan = [FLOOR_RATIO] # [0.2, 0.4, 0.6, 0.8, 1.0]
lowscan = [BARRIER_FLOOR] # np.asarray([0.2]*5)*np.pi
highscan = [BARRIER_HEIGHT] # np.asarray([0.2, 0.4, 0.6, 0.8, 1.0])*np.pi
truth_step_scan = zip(lowscan, highscan)


idx_noise_var = 0 
# Fix to noiseless case
noiseclasses = ['noiseless'] 
noisestrengths = [0.0]
meta_noisevar_scan = zip(noiseclasses, noisestrengths)


# Fix msmt scan - and turn it off!
msmt_per_qubit_scan = [1] # [1, 2, 4, 5, 6, 8, 10, 15, 20, 25, 50]

# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------
# NEW PARAMETER SCANS
# ------------------------------------------------------------------------------

meta_max_iter_scan = [ 5, 10, 15, 20, 25, 50, 75, 100, 125, 250]

lambda_databse = np.load('./lambda_pairs.npz')
lambda1 = list(lambda_databse['lambda_1']) # [0.99, 0.956, 0.922, 0.888, 0.854, 0.820, 0.786, 0.752, 0.718, 0.684, 0.65]
lambda2 = list(lambda_databse['lambda_2']) # [0.977, 0.9752, 0.9734, 0.9716, 0.9698, 0.968, 0.9662, 0.9644, 0.9626, 0.9608, 0.959]
lambda_scan = zip(lambda1, lambda2)

LOOPS_DICT = {"meta_truth_floor_scan": meta_truth_floor_scan,
              "meta_max_iter_scan":meta_max_iter_scan, 
              "meta_noisevar_scan": meta_noisevar_scan,
              "truth_step_scan": truth_step_scan,
              "lambda_scan":lambda_scan,
              "msmt_per_qubit_scan": msmt_per_qubit_scan}

ParamUpdater = DataCube(LOOPS_DICT)


Where to find the './data/' folder:

In [9]:
ls

data_generator.py            lambda_pairs.npz
DATA_preJune2019_SUBMITTED/  risk_data_generator.py
data_v3/                     Risk_Matrix_Generator_datav3.ipynb
job_prefix_0.pbs             Risk_Matrix_Generator_Plots_datav3.ipynb
job_prefix_1.pbs             Single_Run_Plots_datav3.ipynb
job_prefix_2.pbs             Single_Run_Plots_RMS Demo_v3.ipynb
lambda_pairs_2.npz


In [6]:
loadpath = ''

#### Risk score generator script:

In [7]:
# Data variables
max_msmtvar = len(ParamUpdater.meta_max_iter_scan)
max_var = max(len(ParamUpdater.lambda_scan), len(ParamUpdater.msmt_per_qubit_scan))


for prefix in prefix_list:

    # simplify parameter scans by removing loops
    max_prevar_floor = len(ParamUpdater.truth_step_scan) # no truth loop for any prefix
    idx_prevar=0 # no truth loop, as stated above.
    idx_noise_var=0 # no noise strength loop.
    idx_var_dict=0 # only lambda loop, no msmt per iteration loop.

    full_data_matrix = np.zeros((max_prevar_floor, max_msmtvar, max_var, 2, 4))

    for idx_msmt_var in range(max_msmtvar): # loop over max iterations

        macro_performance = []

        for idx_var in range(max_var): # loop over lambda values

            regime_ID = prefix + '_n_' + str(idx_noise_var) +'_vset_' + str(idx_var_dict)
            testcase_ID = regime_ID + '_t_' + str(idx_prevar) + '_m_' + str(idx_msmt_var)

            unique_ID = loadpath + testcase_ID + '_v_' + str(idx_var)

            PATHDICT["fle"] = unique_ID

            perf_metrics = np.zeros((2, 4))

            try:
                qslamdata = np.load(path_to_file(PATHDICT, flag='q'))
                naivedata = np.load(path_to_file(PATHDICT, flag='n'))
            except:
                print "Data filenot loaded. Unique ID:", unique_ID
                macro_performance.append(perf_metrics)
                continue

            perf_metrics[:, 0] = np.asarray([Metric.original_err_metric(data["macro_residuals"]) for data in [qslamdata, naivedata]])
            perf_metrics[:, 1] = np.asarray([Metric.err(data["macro_residuals"]) for data in [qslamdata, naivedata]])
            perf_metrics[:, 2] = np.asarray([Metric.rms(data["macro_residuals"]) for data in [qslamdata, naivedata]])
            perf_metrics[:, 3] = np.asarray([Metric.ssim(data)[0] for data in [qslamdata, naivedata]]) # returning ssim (raw) or deviations

            macro_performance.append(perf_metrics)


        full_data_matrix[idx_prevar, idx_msmt_var, :, :, :] = np.asarray(macro_performance)


    datafile = './data_v3/'+'RISK_' + regime_ID +'.npz'
    np.savez(datafile, full_data_matrix=full_data_matrix)

    print "Made data file:", datafile


Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_0
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_1
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_2
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_3
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_4
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_5
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_6
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_7
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_8
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_9
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_10
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_11
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_12
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0_m_0_v_13
Data filenot loaded. Unique ID: 2019_Jun_1D_n_0_vset_0_t_0